# Multi Layer Percepron
with R kernel

In [14]:
install.packages("tensorflow")


Die heruntergeladenen Bin"arpakete sind in 
	/var/folders/kj/dkjqkk2n3wq2zfbttgdpjrj80000gn/T//Rtmpjw3DfT/downloaded_packages


In [15]:
list_of_packages <- c ("synthpop", "insight", "party", "dplyr", "rpart", "rpart.plot", "randomForest", "pROC", "caret", "pracma", "here", "Hmisc", "arf", "randomForest", "caret", "xgboost", "data.table", "tensorflow","keras")

lapply(list_of_packages, FUN= function(X){
  do.call("require", list(X))
})

Lade n"otiges Paket: tensorflow

Warning message:
"Paket 'tensorflow' wurde unter R Version 4.3.2 erstellt"



Attache Paket: 'tensorflow'


Das folgende Objekt ist maskiert 'package:caret':

    train


Lade n"otiges Paket: keras

Warning message:
"Paket 'keras' wurde unter R Version 4.3.2 erstellt"

Attache Paket: 'keras'


Das folgende Objekt ist maskiert 'package:party':

    fit


Das folgende Objekt ist maskiert 'package:modeltools':

    fit


Das folgende Objekt ist maskiert 'package:insight':

    get_weights




[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[9]]
[1] TRUE

[[10]]
[1] TRUE

[[11]]
[1] TRUE

[[12]]
[1] TRUE

[[13]]
[1] TRUE

[[14]]
[1] TRUE

[[15]]
[1] TRUE

[[16]]
[1] TRUE

[[17]]
[1] TRUE

[[18]]
[1] TRUE

[[19]]
[1] TRUE

## Data

In [3]:
load(file = (paste0(here(), "/cpspop.RData")))
adult <- read.csv(file = (paste0(here(),"/adult_preprocessed.csv")))
# delete NAs
adult[adult == "?"] <- NA
adult <- na.omit(adult)

adult$workclass <- as.factor(adult$workclass)
adult$education <- as.factor(adult$education)
adult$marital_status <- as.factor(adult$marital_status)
adult$relationship <- as.factor(adult$relationship)
adult$race <- as.factor(adult$race)
adult$sex <- as.factor(adult$sex)
adult$native_country <- as.factor(adult$native_country)
adult$income <- as.factor(adult$income)

## Trying out the MLP
1. Prepare Data: Split your data into features and target. Normalize the features to ensure better performance of the neural network. <br>
2. Define and Train the MLP Model: Define the architecture of the MLP, compile the model, and train it on your data. <br>
3. Generate Synthetic Data: Sample from the feature space, use the trained model to predict the target variable for these samples, and combine the predictions with the sampled features to create synthetic data.

In [10]:
# convert all variables in df to numeric
convert_to_numeric <- function(df) {
  # Convert factors to numeric
  df[] <- lapply(df, function(col) {
    if (is.factor(col)) {
      return(as.numeric(col))
    } else {
      return(col)
    }
})
  return(df)
}

In [18]:
# Identify features and target
data <- adult

data <- convert_to_numeric(data)
target_col <- 'income'
features <- setdiff(names(data), target_col)

# Split the data into features (X) and target (y)
X <- data[, features]
y <- data[, target_col]

X_means <- colMeans(X)
X_sds <- apply(X, 2, sd)

# Normalize features
X <- scale(X)

In [19]:
# Define the MLP model
model <- keras_model_sequential() %>%
  layer_dense(units = 64, activation = 'relu', input_shape = ncol(X)) %>%
  layer_dense(units = 32, activation = 'relu') %>%
  layer_dense(units = 1)  # Single output for regression

# Compile the model
model %>% compile(
  loss = 'mean_squared_error',
  optimizer = optimizer_adam(),
  metrics = c('mean_absolute_error')
)

# Train the model
history <- model %>% fit(
  as.matrix(X), y,
  epochs = 100,
  batch_size = 32,
  validation_split = 0.2
)

In [ ]:
generate_synthetic_data <- function(model, X, X_means, X_sds, n_samples) {
  synthetic_data <- list()
  for (i in 1:n_samples) {
    # Sample a random row from the normalized feature matrix
    random_sample <- X[sample(1:nrow(X), 1), , drop = FALSE]
    
    # Predict the target variable for the sampled row
    prediction <- model %>% predict(as.matrix(random_sample))
    
    # Combine the random sample with the predicted target
    synthetic_sample <- as.data.frame(random_sample)
    synthetic_sample$income <- prediction
    
    # Denormalize the features and the target
    denormalized_sample <- synthetic_sample
    for (col in names(synthetic_sample)[-ncol(synthetic_sample)]) {
      denormalized_sample[[col]] <- synthetic_sample[[col]] * X_sds[col] + X_means[col]
    }
    
    # Append to the list
    synthetic_data[[i]] <- denormalized_sample
  }
  
  # Combine all synthetic samples into a data frame
  synthetic_df <- do.call(rbind, synthetic_data)
  return(synthetic_df)
}


             age  workclass     fnlwgt  education marital_status occupation
5422  -1.0992212 -0.2089512 -0.2341613 -0.6121683     -0.3872683 -0.9826988
28285 -1.2514900 -0.2089512  2.3773602  1.2239975      0.9478313  0.2581295
6276  -0.4140115 -0.2089512 -0.4328306 -0.3498589     -0.3872683 -0.7345332
27027 -0.9469524 -1.2572521  0.8842455  1.2239975      0.9478313  1.0026265
22895  1.4893489  1.8876507 -1.4811210  0.1747599      2.2829308 -0.4863675
24891  0.4234671 -1.2572521  0.2164744  1.2239975     -0.3872683  1.0026265
      relationship       race        sex capital_gain capital_loss
5422    -0.8857223  0.3850415  0.6927947   -0.1474422   -0.2185824
28285    0.9877108  0.3850415 -1.4433813   -0.1474422   -0.2185824
6276    -0.8857223  0.3850415  0.6927947   -0.1474422   -0.2185824
27027    0.9877108  0.3850415  0.6927947   -0.1474422   -0.2185824
22895   -0.2612446  0.3850415  0.6927947   -0.1474422   -0.2185824
24891   -0.8857223 -2.0110019  0.6927947   -0.1474422   -0.2185824

In [ ]:
# Generate synthetic data
synthetic_data <- generate_synthetic_data(model, X_normalized, X_means, X_sds, 1000)  # Generate 1000 synthetic samples

print(head(synthetic_data))